In [1]:
import os, glob
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from pyBioInfo.IO.File import GtfFile, GtfTranscriptBuilder, FamFile, BedFile
from pyBioInfo.Utils import ShiftLoader
infos = pd.read_excel("data/NASCseq.xlsx")

In [7]:
infos = pd.read_excel("NASCseq_filtered.xls", sep="\t")

data = dict()

rs = ["20220113_NASCseq_K562", "20220321_NASCseq_K562", "20220418_NASCseq_K562"]
tmp = infos[[x in rs for x in infos["Run"]]]
tmp = tmp[(tmp["4sU"] == 50) & (tmp["Time"] == 3)]
print(len(tmp))
data["NASC-seq"] = tmp

tmp = infos[[x in ["GSE128273_NASCseq_K562_SE"] for x in infos["Run"]]]
tmp = tmp[(tmp["4sU"] == 50) & (tmp["Time"] == 3)]
print(len(tmp))
data["GSE128273"] = tmp

28
67


In [4]:
data = dict()

rs = ["20220113_NASCseq_K562", "20220321_NASCseq_K562", "20220418_NASCseq_K562"]
tmp = infos[[x in rs for x in infos["Run"]]]
tmp = tmp[(tmp["4sU"] == 50) & (tmp["Time"] == 3)]
print(len(tmp))
data["NASC-seq"] = tmp

tmp = infos[[x in ["GSE128273_NASCseq_K562_SE"] for x in infos["Run"]]]
tmp = tmp[(tmp["4sU"] == 50) & (tmp["Time"] == 3)]
print(len(tmp))
data["GSE128273"] = tmp

38
80


In [35]:
conversions = dict()
for k, v in data.items():
    for run, cell in v[["Run", "Cell"]].values:
        path = "results/mismatch/report/%s/%s/ratio.ce.tsv" % (run, cell)
        d = pd.read_csv(path, sep="\t", index_col=0)
        v = d.loc["TC"]["Ratio"]
        conversions[cell] = v

In [23]:
fpkms = dict()
for k, v in data.items():
    for run, cell in v[["Run", "Cell"]].values:
        path = "results/nascent/expression/annotated/%s/%s.tsv" % (run, cell)
        fpkms[cell] = pd.read_csv(path, sep="\t")

In [53]:
merged = dict()
for k, v in data.items():
    ref = None
    n = 0
    for run, cell in v[["Run", "Cell"]].values:
        d = fpkms[cell]
        if d["LibSize"].values[0] > 500000 and conversions[cell] > 0.008:
            n += 1
            if ref is None:
                ref = d.copy()
            else:
                ref["Count"] = ref["Count"] + d["Count"]
                ref["Count[Exists]"] = ref["Count[Exists]"] + d["Count[Exists]"]
                ref["Count[Nascent]"] = ref["Count[Nascent]"] + d["Count[Nascent]"]
                ref["LibSize"] = ref["LibSize"] + d["LibSize"]
    print(n)
    ref["FPKM"] = ref["Count"] * 1e9 / ref["Length"] / ref["LibSize"]
    ref["FPKM[Exists]"] = ref["Count[Exists]"] * 1e9 / ref["Length"] / ref["LibSize"]
    ref["FPKM[Nascent]"] = ref["Count[Nascent]"] * 1e9 / ref["Length"] / ref["LibSize"]
    
    Ts = []
    t = 3
    for total, nascent in ref[["Count", "Count[Nascent]"]].values:
        T = t / (np.log2(total) - np.log2(total - nascent))
        Ts.append(T)
    ref["T"] = Ts
    merged[k] = ref

30


/home/chenzonggui/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log2
/home/chenzonggui/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
/home/chenzonggui/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in double_scalars


64


In [52]:
tids = ["ENST00000652288.1", "ENST00000317968.9", "ENST00000229239.10"]
gnames = ["MYC", "PDLIM5", "GAPDH"]

for k, d in merged.items():
    print("-" * 80)
    for tid, gname in zip(tids, gnames):
        d1 = d[d["TranscriptID"] == tid]
        v1 = d1["Count"].values[0]
        v2 = d1["Count[Nascent]"].values[0]
        print(gname, v1, v2, v2 / v1, sep="\t")

--------------------------------------------------------------------------------
MYC	1798	1564	0.8698553948832035
PDLIM5	3461	997	0.28806703264952327
GAPDH	178123	17122	0.09612458806555021
--------------------------------------------------------------------------------
MYC	5741	3937	0.6857690297857516
PDLIM5	5290	1139	0.21531190926275992
GAPDH	481689	27296	0.05666726871487619


In [50]:
for k, d in merged.items():
    path = "results/halflife/%s_K562_50uM_3h.tsv" % k
    d.to_csv(path, sep="\t", index=False)